In [1]:
# Import the required libraries and dependencies
import numpy as np
import pandas as pd
import os
import requests
import json
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from dotenv import load_dotenv
#from MCForecastTools import MCSimulation

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

#Add disclaimer for efficacy of model (restricted to CONUS)

In [2]:
#Read CSV file from the Resources folder into a Pandas Dataframe

realtor_data = pd.read_csv(
    Path("Resources/realtor_data.csv")
        )
    
#Review the dataframe 
realtor_data

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
0,for_sale,105000.0,3.0,2.0,0.12,"Sector Yahuecas Titulo # V84, Adjuntas, PR, 00601",Sector Yahuecas Titulo # V84,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,for_sale,80000.0,4.0,2.0,0.08,"Km 78 9 Carr # 135, Adjuntas, PR, 00601",Km 78 9 Carr # 135,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,for_sale,67000.0,2.0,1.0,0.15,"556G 556-G 16 St, Juana Diaz, PR, 00795",556G 556-G 16 St,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,for_sale,145000.0,4.0,2.0,0.10,"R5 Comunidad El Paraso Calle De Oro R-5 Ponce,...",R5 Comunidad El Paraso Calle De Oro R-5 Ponce,Ponce,Puerto Rico,731.0,1800.0,NaN
4,for_sale,65000.0,6.0,2.0,0.05,"14 Navarro, Mayaguez, PR, 00680",14 Navarro,Mayaguez,Puerto Rico,680.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
402154,for_sale,299900.0,4.0,3.0,0.29,"299 Evening Star Dr, Naugatuck, CT, 06770",299 Evening Star Dr,Naugatuck,Connecticut,6770.0,2374.0,2017-06-27
402155,for_sale,275000.0,3.0,2.0,0.53,"570 Witches Rock Rd, Bristol, CT, 06010",570 Witches Rock Rd,Bristol,Connecticut,6010.0,1548.0,2002-06-17
402156,for_sale,99900.0,2.0,1.0,0.11,"335 Raymond St, Waterbury, CT, 06706",335 Raymond St,Waterbury,Connecticut,6706.0,569.0,NaN
402157,for_sale,259700.0,3.0,2.0,0.83,"14 S Main St, East Granby, CT, 06026",14 S Main St,East Granby,Connecticut,6026.0,1656.0,NaN


In [3]:
#Review categories
realtor_data.columns
#Notes: inputs
#bed, bath, acre, state, house_size 
#target: price

Index(['status', 'price', 'bed', 'bath', 'acre_lot', 'full_address', 'street',
       'city', 'state', 'zip_code', 'house_size', 'sold_date'],
      dtype='object')

In [4]:
# Drop unncessary data
realtor_data = realtor_data.drop(columns = ['status', 'full_address', 'street', 'city', 'zip_code', 'sold_date'])
#Review Dataframe
realtor_data

,price,bed,bath,acre_lot,state,house_size
0,105000.0,3.0,2.0,0.12,Puerto Rico,920.0
1,80000.0,4.0,2.0,0.08,Puerto Rico,1527.0
2,67000.0,2.0,1.0,0.15,Puerto Rico,748.0
3,145000.0,4.0,2.0,0.10,Puerto Rico,1800.0
4,65000.0,6.0,2.0,0.05,Puerto Rico,NaN
...,...,...,...,...,...,...
402154,299900.0,4.0,3.0,0.29,Connecticut,2374.0
402155,275000.0,3.0,2.0,0.53,Connecticut,1548.0
402156,99900.0,2.0,1.0,0.11,Connecticut,569.0
402157,259700.0,3.0,2.0,0.83,Connecticut,1656.0


In [5]:
#realtor_data['sold_date']=pd.to_datetime(realtor_data['sold_date'], infer_datetime_format=True)
#realtor_data['sold_date'].dt.year
#realtor_data = realtor_data[realtor_data['sold_date'].notna()]
#realtor_data['sold_date'].isna().sum()

In [6]:
#Plot Data
#realtor_data.groupby([realtor_data['sold_date'].dt.year, realtor_data['sold_date'].dt.month])['price'].mean().plot(figsize=(15, 5))

In [7]:
realtor_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402159 entries, 0 to 402158
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   price       402159 non-null  float64
 1   bed         314887 non-null  float64
 2   bath        316400 non-null  float64
 3   acre_lot    326858 non-null  float64
 4   state       402159 non-null  object 
 5   house_size  318977 non-null  float64
dtypes: float64(5), object(1)
memory usage: 18.4+ MB


In [8]:
#Counting number of states 
realtor_data['state'].value_counts()

Massachusetts     173162
New Hampshire      51394
Vermont            46460
Maine              36650
Connecticut        32768
Rhode Island       28518
Puerto Rico        24679
New York            5828
Virgin Islands      2573
Georgia               48
South Carolina        24
Tennessee             20
Virginia              20
New Jersey            12
Wyoming                3
Name: state, dtype: int64

In [9]:
#Transform categorical variables

from sklearn.preprocessing import OneHotEncoder
#Create Instance
enc = OneHotEncoder(handle_unknown='ignore')
#Fit instance to categorical data
enc.fit(realtor_data[['state']])

#Transform data and create DataFrame
categories = enc.transform(realtor_data[['state']])

#Create Pandas Dataframe

lorem = pd.DataFrame(categories.toarray(),columns = enc.categories_)
#Review dataframe
display(lorem)

#pd.DataFrame(enc.fit_transform(realtor_data[['states']]).toarray())

#drop_enc = pd.DataFrame(OneHotEncoder().fit(realtor_data["state"].value_counts()))

#enc = OneHotEncoder(handle_unknown='ignore')
#enc_df = pd.DataFrame(enc.fit_transform(bridge_df[['Bridge_Types_Cat']]).toarray())


,Connecticut,Georgia,Maine,Massachusetts,New Hampshire,New Jersey,New York,Puerto Rico,Rhode Island,South Carolina,Tennessee,Vermont,Virgin Islands,Virginia,Wyoming
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402154,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402155,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402156,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402157,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
#Review categories

enc.categories_

[array(['Connecticut', 'Georgia', 'Maine', 'Massachusetts',
        'New Hampshire', 'New Jersey', 'New York', 'Puerto Rico',
        'Rhode Island', 'South Carolina', 'Tennessee', 'Vermont',
        'Virgin Islands', 'Virginia', 'Wyoming'], dtype=object)]

In [11]:
#Omit Puerto Rico and Virigin Islands to avoid Colinearity issues
lorem = lorem.drop(columns = ['Virgin Islands'])

display(lorem)

,Connecticut,Georgia,Maine,Massachusetts,New Hampshire,New Jersey,New York,Puerto Rico,Rhode Island,South Carolina,Tennessee,Vermont,Virginia,Wyoming
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402154,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402155,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402156,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402157,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
#Concatenate new dataframe to exisitng dataframe

realtor_states_enc = pd.concat([realtor_data,lorem], axis = 1)

#Drop states column
realtor_states_enc = realtor_states_enc.drop(columns = ['state'])

#Review Dataframe
realtor_states_enc.head(10)

,price,bed,bath,acre_lot,house_size,"(Connecticut,)","(Georgia,)","(Maine,)","(Massachusetts,)","(New Hampshire,)","(New Jersey,)","(New York,)","(Puerto Rico,)","(Rhode Island,)","(South Carolina,)","(Tennessee,)","(Vermont,)","(Virginia,)","(Wyoming,)"
0,105000.0,3.0,2.0,0.12,920.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,80000.0,4.0,2.0,0.08,1527.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,67000.0,2.0,1.0,0.15,748.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,145000.0,4.0,2.0,0.10,1800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,65000.0,6.0,2.0,0.05,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,179000.0,4.0,3.0,0.46,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,50000.0,3.0,1.0,0.20,2040.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,71600.0,3.0,2.0,0.08,1050.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,100000.0,2.0,1.0,0.09,1092.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,300000.0,5.0,3.0,7.46,5403.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#Drop null values
realtor_states_enc = realtor_states_enc.dropna()

In [14]:
#Review Data to ensure dataframe is ready for training model
realtor_states_enc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239772 entries, 0 to 402158
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   price              239772 non-null  float64
 1   bed                239772 non-null  float64
 2   bath               239772 non-null  float64
 3   acre_lot           239772 non-null  float64
 4   house_size         239772 non-null  float64
 5   (Connecticut,)     239772 non-null  float64
 6   (Georgia,)         239772 non-null  float64
 7   (Maine,)           239772 non-null  float64
 8   (Massachusetts,)   239772 non-null  float64
 9   (New Hampshire,)   239772 non-null  float64
 10  (New Jersey,)      239772 non-null  float64
 11  (New York,)        239772 non-null  float64
 12  (Puerto Rico,)     239772 non-null  float64
 13  (Rhode Island,)    239772 non-null  float64
 14  (South Carolina,)  239772 non-null  float64
 15  (Tennessee,)       239772 non-null  float64
 16  (V

## PART 2: Fit the Model

In [15]:
#Target
y = realtor_states_enc[['price']]
#Inputs
X = realtor_states_enc.drop(columns = y)

#Review Data
display(y)
display(X)
#X['state'].value_counts()

,price
0,105000.0
1,80000.0
2,67000.0
3,145000.0
5,179000.0
...,...
402154,299900.0
402155,275000.0
402156,99900.0
402157,259700.0


,bed,bath,acre_lot,house_size,"(Connecticut,)","(Georgia,)","(Maine,)","(Massachusetts,)","(New Hampshire,)","(New Jersey,)","(New York,)","(Puerto Rico,)","(Rhode Island,)","(South Carolina,)","(Tennessee,)","(Vermont,)","(Virginia,)","(Wyoming,)"
0,3.0,2.0,0.12,920.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,2.0,0.08,1527.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,1.0,0.15,748.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,2.0,0.10,1800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.46,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402154,4.0,3.0,0.29,2374.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402155,3.0,2.0,0.53,1548.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402156,2.0,1.0,0.11,569.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402157,3.0,2.0,0.83,1656.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from sklearn.model_selection import train_test_split

#Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [17]:
from sklearn.linear_model import LinearRegression

#Instantiate Regression Model
model = LinearRegression().fit(X,y,sample_weight = None)

In [18]:
#Prediction Scores
model.score(X_train, y_train)

0.26749836212042555

In [19]:
#Validate Prediction Scores
model.score(X_test, y_test)

0.18104192597116375

In [20]:
#Steps for improving models
#1. random Forrest Model
#2. Try dropping unimportant inputs
#3. Modify inputs 
#4. Test for multicollinearity 
#(sklearn, VIF>10 => Drop Something --implies high colinearity, VIF< 5, reasonable)
#5. Refine dataset by adding zip code (1HCE)
#6. Encapsulate zip code with median house price
#7. 30 year fixed mortage rate (maybe)

In [21]:
#Adjust for inflation
#Lorem Ipsum

In [22]:
## Load the environment variables from the .env file
#by calling the load_dotenv function
#STORE API KEY IN .ENV FILE

#load_dotenv()

In [23]:
#Make the API Calls
# Make the API request
#response = requests.get("endpoint").json()

# Display the response data
#print(json.dumps(response, indent=4, sort_keys=True))